In [1]:
role = "YOUR_AWS_KEYS"
bucket_name = "llama-training-s3"
s3_prefix = "llama-training-s3/quantized"
s3_output_path=f"s3://{bucket_name}/{s3_prefix}/model"
hf_token="YOUR_HUGGINGFACE_KEYS"

In [2]:
!pip install awscli --force-reinstall --upgrade

^C


In [4]:
!pip install sagemaker

  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached botocore-1.35.99-py3-none-any.whl.metadata (5.7 kB)
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.17-py39-none-any.whl.metadata (7.2 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
Using cached botocore-1.35.99-py3-none-any.whl (13.3 MB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached mock-4.0.3-py3-none-any.whl (28 kB)
Using cached multiprocess-0.70.17-py39-none-any.whl (133 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
doccano 1.8.4 requires chardet<5.0.0,>=4.0.0, which is not installed.
doccano 1.8.4 requires SQLAlchemy<2.0.0,>=1.4.31, which is not installed.
matplotlib 3.8.3 requires cycler>=0.10, which is not installed.
matplotlib 3.8.3 requires fonttools>=4.22.0, which is not installed.
matplotlib 3.8.3 requires kiwisolver>=1.3.1, which is not installed.
matplotlib 3.8.3 requires pillow>=8, which is not installed.
matplotlib 3.8.3 requires pyparsing>=2.3.1, which is not installed.
mlflow 2.17.1 requires Flask<4, which is not installed.
mlflow 2.17.1 requires markdown<4,>=3.3, which is not installed.
mlflow 2.17.1 requires sqlalchemy<3,>=1.4.0, which is not installed.
poetry 1.5.1 requires pexpect<5.0.0,>=4.7.0, which is not installed.
ray 2.4.0 requires grpcio<=1.51.3,>=1.32.0; python_version < "3.10" and sys_platform != "da

In [3]:
!pip install boto3 

^C



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
!explorer .

In [16]:
%mkdir training_code

Ya existe el subdirectorio o el archivo training_code.


In [2]:
%%writefile training_code/quantize_llm.py

import subprocess
subprocess.run(["pip", "install", "autoawq", "boto3"])

import os
import torch
import argparse
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import boto3

role = "YOUR_AWS_KEYS"
bucket_name = "llama-training-s3"
s3_prefix = "llama-training-s3/quantized"



# Parse arguments (S3 bucket and model path)
parser = argparse.ArgumentParser()
parser.add_argument("--model_path", type=str, default="javiagu/KULLM3_pCliNER_bf16_Official_Official")
parser.add_argument("--quant_path", type=str, default="/opt/ml/model/KULLM3_pCliNER_bf16_Official_Official")
parser.add_argument("--s3_output", type=str, required=True, help="S3 path to upload quantized model")
parser.add_argument("--hf_token", type=str, required=True, help="Hugging Face authentication token")

args = parser.parse_args()

# Define quantization config
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

print(f"Loading model from {args.model_path}...")
model = AutoAWQForCausalLM.from_pretrained(
    args.model_path, low_cpu_mem_usage=True, use_cache=False, torch_dtype=torch.float16, use_auth_token=args.hf_token
)
tokenizer = AutoTokenizer.from_pretrained(args.model_path, use_auth_token=args.hf_token, trust_remote_code=True)

# Perform quantization
print("Quantizing model...")
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model locally
print(f"Saving quantized model at {args.quant_path}...")
model.save_quantized(args.quant_path)
tokenizer.save_pretrained(args.quant_path)

# Upload to S3 for downloading later
print(f"Uploading quantized model to {args.s3_output}...")
s3_client = boto3.client("s3")
bucket_name, prefix = args.s3_output.replace("s3://", "").split("/", 1)

for root, _, files in os.walk(args.quant_path):
    for file in files:
        local_path = os.path.join(root, file)
        s3_path = os.path.join(prefix, os.path.relpath(local_path, args.quant_path))
        print(f"Uploading {local_path} to s3://{bucket_name}/{s3_prefix}")
        s3_client.upload_file(local_path, bucket_name, s3_prefix)

print("Quantization complete and uploaded to S3.")


Overwriting training_code/quantize_llm.py


In [3]:
%%writefile training_code/requirements.txt
boto3
transformers
torch
datasets
accelerate
sentencepiece
bitsandbytes
peft
pyarrow
deepspeed
accelerate
autoawq

Overwriting training_code/requirements.txt


In [4]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    entry_point="quantize_llm.py",
    source_dir="training_code",
    role=role,
    instance_count=1,
    instance_type="ml.g5.4xlarge",  # Change based on needs
    framework_version="2.5.1",
    py_version="py311",
    dependencies=["requirements.txt"],
    hyperparameters={
        "s3_output": s3_output_path,
        "hf_token": hf_token  # Add your Hugging Face token here
    },
    output_path=s3_output_path
)

# Fit the model
estimator.fit()


C:\Users\javia\anaconda3\envs\aws\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/06/25 09:05:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=237707;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=518156;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\javia\AppData\Local\sagemaker\sagemaker\config.yaml


[03/06/25 09:05:12] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=453644;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=185354;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[03/06/25 09:05:13] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=386005;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=206855;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[03/06/25 09:05:14] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=48832;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=264535;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=448969;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=534227;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[03/06/25 09:05:16] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=125774;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=71649;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=168915;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=494452;file://C:\Users\javia\anaconda3\envs\aws\lib\site-packages\sagemaker\session.py#1042\1042]8;;\
                             pytorch-training-2025-03-06-00-05-14-843                                              

2025-03-06 00:05:22 Starting - Starting the training job
......3-06 00:05:22 Pending - Training job waiting for capacity
...5-03-06 00:06:08 Pending - Preparing the instances for training
.....................ownloading - Downloading the training image
...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.127.05
Current installed NVIDIA driver version is 550.144.03
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-03-06 00:11:52,826 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-03-06 00:11:52,844 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-03-06 00:11:52,853 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-03-06 00:11:52,854 sagemaker_pytorch_container.training INFO     Invoking user training

In [ ]:
###KULLM3 AWQ is 04-324

In [ ]:
import boto3

s3 = boto3.client("s3")
bucket = "your-bucket"
prefix = "quantized_model/"

# Download all files
for obj in s3.list_objects_v2(Bucket=bucket, Prefix=prefix).get("Contents", []):
    file_name = obj["Key"].split("/")[-1]
    s3.download_file(bucket, obj["Key"], f"./quantized_model/{file_name}")
